In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC
import time
import os
import gc
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mutual_info_score, precision_score, recall_score, fbeta_score, confusion_matrix, accuracy_score
import lightgbm as lgb
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [ ]:
test=pd.read_csv('diabete_test.csv' )
train=pd.read_csv('diabete_train.csv' )
print(test.shape)
print(train.shape)

In [ ]:
y_train=train['diabetesMed']
y_test=test['diabetesMed']
train_df=train.drop(['diabetesMed'], axis=1 , inplace=True)
test_df=test.drop(['diabetesMed'], axis=1 , inplace=True)

# Random forest

In [ ]:
a = []
b = []
res = []
for i in range(30):  
    features = train_df.columns
   

    importances = pd.DataFrame()
    folds = KFold(n_splits=10, shuffle=True, random_state=1)
    clfs = []
    averged = []
    oof_preds = np.zeros([train_df.shape[0], 2])
    oof_c_preds = np.zeros(train_df.shape[0])
    error = []
    
    
    for fold_, (trn_, val_) in enumerate(folds.split(y_train, y_train)):
        trn_x, trn_y = train_df[features].iloc[trn_], y_train.iloc[trn_]
        val_x, val_y = train_df[features].iloc[val_], y_train.iloc[val_]

        temp = []

        clf = RandomForestClassifier(n_estimators=300 ,criterion='gini' ,max_depth=5 )
        model=clf.fit(trn_x, trn_y)

        oof_preds[val_, :] = clf.predict_proba(val_x)
        oof_c_preds[val_] = clf.predict(val_x)
        print(fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(precision_score(pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(recall_score(pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(accuracy_score(y_pred=clf.predict(val_x),
                          y_true=val_y))
        print('Fold AUC :', roc_auc_score(val_y, clf.predict_proba(val_x)[:, 1]))
        


#         res.append([fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), precision_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), recall_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y),
#                           accuracy_score(y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1])])
       
    clfs.append(clf)
    imp_df = pd.DataFrame()
    imp_df['feature'] = features
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    print('OOF AUC :', roc_auc_score(y_train, oof_preds[:, 1]))
    res.append([fbeta_score(beta=1, pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), precision_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), recall_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train),
                          accuracy_score(y_pred=oof_c_preds,
                          y_true=y_train), roc_auc_score(y_train, oof_preds[:, 1])])
    
    #print(oof_preds[:, 1])
    #print(res)
    test_predictions = np.zeros([test_df.shape[0], 2])
    for clf in clfs:
        test_predictions += clf.predict_proba(test_df[features])
    test_predictions = test_predictions / 5
    

In [ ]:
res_df_rf = pd.DataFrame(res)
res_df_rf.columns = ['f_1', 'precision', 'recall', 'accuracy', 'AUC']
res_df_rf

In [ ]:
RF_2_=importances.sort_values(by=['gain'],ascending=False, inplace=False)
RF_2_

# LGBM

In [ ]:
ws={0:0.5,1:3}
a = []
b = []
res = []
for i in range(30):
    features=X.columns
    importances = pd.DataFrame()
    folds = KFold(n_splits=10, shuffle=True, random_state=1)
    clfs = []
    averged = []
    oof_preds = np.zeros([train_df.shape[0], 2])
    oof_c_preds = np.zeros(train_df.shape[0])
    error = []
    lgb_params = {
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.15,
    'reg_alpha': .01,
    'reg_lambda': 1,
    'min_split_gain': 0.01,
    'min_child_weight': 5,
    'n_estimators': 4000,
    'bagging_fraction': 0.65,
    'bagging_freq': 5,
    'silent': -1,
    'is_unbalalnced': False,
    'num_leaves':800,
    'verbose': -1,
    'scale_pos_weight':3.5,
    'max_depth': 8}

    
    
    for fold_, (trn_, val_) in enumerate( folds.split(y_train, y_train)):
        trn_x, trn_y = train_df[features].iloc[trn_], y_train.iloc[trn_]
        val_x, val_y = train_df[features].iloc[val_], y_train.iloc[val_]

        temp = []

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(val_x, val_y)],
            verbose=100,
            sample_weight=trn_y.map(ws),
            early_stopping_rounds=200
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        oof_c_preds[val_] = clf.predict(val_x, num_iteration=clf.best_iteration_)
        print(fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
                          y_true=val_y))
        print(precision_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
                          y_true=val_y))
        print(recall_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
                          y_true=val_y))
        print(accuracy_score(y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
                          y_true=val_y))
        print('Fold AUC :', roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
        

#         res.append([fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), precision_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), recall_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y),
#                           accuracy_score(y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1])])
       
#         clfs.append(clf)
    imp_df = pd.DataFrame()
    imp_df['feature'] = features
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    print('OOF AUC :', roc_auc_score(y_train, oof_preds[:, 1]))
    res.append([fbeta_score(beta=1, pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), precision_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), recall_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train),
                          accuracy_score(y_pred=oof_c_preds,
                          y_true=y_train), roc_auc_score(y_train, oof_preds[:, 1])])
    #print(oof_preds[:, 1])
    #print(res)
    test_predictions = np.zeros([test_df.shape[0], 2])
    for clf in clfs:
        test_predictions += clf.predict_proba(test_df[features])
    test_predictions = test_predictions / 10
    


In [ ]:
res_df = pd.DataFrame(res)
res_df.columns = ['f_1', 'precision', 'recall', 'accuracy', 'AUC']
res_df

In [ ]:
importances.sort_values(by=['gain'],ascending=False, inplace=True)
#importances.to_csv('lgbm.csv')
importances

# SVM

In [ ]:

a = []
b = []
res = []
for i in range(30):  
    features = train_df.columns

    importances = pd.DataFrame()
    folds = KFold(n_splits=5, shuffle=True, random_state=1)
    clfs = []
    averged = []
    oof_preds = np.zeros([train_df.shape[0], 2])
    oof_c_preds = np.zeros(train_df.shape[0])
    error = []

    
    for fold_, (trn_, val_) in enumerate(folds.split(y_train, y_train)):
        trn_x, trn_y = train_df[features].iloc[trn_], y_train.iloc[trn_]
        val_x, val_y = train_df[features].iloc[val_], y_train.iloc[val_]

        temp = []

        clf = SVC(C=0.5 , kernel='rbf' ,gamma='auto_deprecated', probability=True)
        clf.fit(trn_x, trn_y)

        oof_preds[val_, :] = clf.predict_proba(val_x)
        oof_c_preds[val_] = clf.predict(val_x)
        print(fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(precision_score(pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(recall_score(pos_label=1, y_pred=clf.predict(val_x),
                          y_true=val_y))
        print(accuracy_score(y_pred=clf.predict(val_x),
                          y_true=val_y))
        print('Fold AUC :', roc_auc_score(val_y, clf.predict_proba(val_x)[:, 1]))
        
        del clf

#         res.append([fbeta_score(beta=1, pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), precision_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), recall_score(pos_label=1, y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y),
#                           accuracy_score(y_pred=clf.predict(val_x, num_iteration=clf.best_iteration_),
#                           y_true=val_y), roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1])])
       
#         clfs.append(clf)
#         imp_df = pd.DataFrame()
#         imp_df['feature'] = features
#         imp_df['gain'] = clf.feature_importances_
#         imp_df['fold'] = fold_ + 1
#         importances = pd.concat([importances, imp_df], axis=0, sort=False)
    print('OOF AUC :', roc_auc_score(y_train, oof_preds[:, 1]))
    res.append([fbeta_score(beta=1, pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), precision_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train), recall_score(pos_label=1, y_pred=oof_c_preds,
                          y_true=y_train),
                          accuracy_score(y_pred=oof_c_preds,
                          y_true=y_train), roc_auc_score(y_train, oof_preds[:, 1])])
    #print(oof_preds[:, 1])
    #print(res)
    test_predictions = np.zeros([test_df.shape[0], 2])
    for clf in clfs:
        test_predictions += clf.predict_proba(test_df[features])
    test_predictions = test_predictions / 5
    

In [ ]:
res_df_svm = pd.DataFrame(res)
res_df_svm.columns = ['f_1', 'precision', 'recall', 'accuracy', 'AUC']
res_df_svm